In [1]:
# Imports

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import torch
from torch import nn
import pyro
import pyro.distributions as dist
from pyro.infer import SVI, TraceEnum_ELBO, Predictive, NUTS, MCMC, config_enumerate
from pyro.infer.autoguide import AutoDelta, AutoDiagonalNormal, AutoMultivariateNormal
from pyro.optim import Adam, ClippedAdam
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from pyro.distributions import MultivariateNormal as MN

In [2]:
# Load data files

X_train_tensor, X_val_tensor, X_test_tensor =   torch.load("./data/X_train_tensor.pt"), \
                                                torch.load("./data/X_val_tensor.pt"),   \
                                                torch.load("./data/X_test_tensor.pt")
U_train_tensor, U_val_tensor, U_test_tensor =   torch.load("./data/U_train_tensor.pt"), \
                                                torch.load("./data/U_val_tensor.pt"),   \
                                                torch.load("./data/U_test_tensor.pt")
N_t_train, N_t_valid, N_t_test = np.load("./data/N_t_train.npy"), \
                                 np.load("./data/N_t_val.npy"),   \
                                 np.load("./data/N_t_test.npy")


In [3]:
# Class for extracting data

class RFNDataset(Dataset):
    """Spatio-temporal demand modelling dataset."""
    def __init__(self, X, U):
        self.X = X
        self.U = U

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()
        
        X_i, U_i = self.X[idx].float(), self.U[idx].float()
        return X_i, U_i

In [38]:
# Recurrent Mixture Density network

class RMDN(nn.Module):
    """
    Pure PyTorch class for Recurrent Mixture Density Network
    Inputs:
        input_dim:  dimension of input tensor U
        hidden_dim: number of hidden units to be used in various hidden layers
        output_dim: dimension of output, lat/lon
        K: number of mixture components
        
    Outputs:
        loc:    Tensor of mean values for Gaussians
        pi:     Mixture components
        Cov:    Covariance matrices
        hidden: Hidden states
    
    """
    def __init__(self, input_dim, hidden_dim, output_dim, K=10):
        super(RMDN, self).__init__()
        # Define parameters
        self.input_dim = input_dim
        self.hidden_dim= hidden_dim
        self.output_dim= output_dim
        self.K = K 
        self.tril_indices = torch.tril_indices(row=output_dim, col=output_dim, offset=-1)

        # Define LSTM
        self.lstm = nn.LSTM(input_size=self.input_dim,
                            hidden_size=self.hidden_dim,
                            num_layers=1)
        # Fully connected layer 
        self.lstm_to_hidden     = nn.Linear(in_features=self.hidden_dim, out_features=self.hidden_dim)
        # Take output of fully connected layer and feed to layers for GMM components
        self.hidden_to_loc      = nn.Linear(in_features=self.hidden_dim, out_features=self.K*self.output_dim)
        self.hidden_to_sigma    = nn.Linear(in_features=self.hidden_dim, out_features=self.K*self.output_dim)
        self.hidden_to_off_diag = nn.Linear(in_features=self.hidden_dim, out_features=self.K)
        self.hidden_to_mix      = nn.Linear(in_features=self.hidden_dim, out_features=self.K)
        
        # Functions
        self.relu = nn.ReLU()
        self.softmax = nn.Softmax(dim=1)
        self.softplus = nn.Softplus()
        
    def forward(self, U, hidden):     
        # Feed through LSTM
        y, hidden = self.lstm(U.view(-1, 1, self.input_dim), hidden)
        # Fully connected
        y = self.relu(self.lstm_to_hidden(y))
        # Compute mean values
        loc   = self.hidden_to_loc(y).view(-1, self.K, self.output_dim)
        # Compute variances (must be positive)
        sigma = self.softplus(self.hidden_to_sigma(y)).view(-1, self.K, self.output_dim)
        # Compute covariances
        cov   = self.hidden_to_off_diag(y).view(-1, self.K, 1)
        # Compute mixture components (must sum to 1)
        pi    = self.softmax(self.hidden_to_mix(y).view(-1,self.K, 1))
        # Create full covariance matrix
        cov_tril = torch.zeros((U.shape[0], self.K, self.output_dim, self.output_dim))
        for i in range(self.K):
            cov_tril[:, i, self.tril_indices[0], self.tril_indices[1]] = cov[:, i, :]
            cov_tril[:, i] += torch.diag_embed(sigma[:, i, :])
        
        return (loc, pi, cov_tril), hidden
    
    def init_hidden(self):
        # Initialise hidden states
        return (torch.zeros(1, 1, self.hidden_dim), torch.zeros(1, 1, self.hidden_dim))
    
    def get_loglikelihood(self, U, X, mask):
        
        logprob = 0
        T_max = U.size(1)
        with torch.no_grad():
            hidden = self.init_hidden()
            for t in range(0, T_max):
                # Extract components for current GMM
                (loc, pi, cov), hidden = self.forward(U=U[:, t, :, :], hidden=hidden)
                # Compute loglikelihood for all datapoints for current time interval
                for tt in range(mask[t]):
                    current_prob = 0
                    for i in range(self.K):
                        current_prob += pi.squeeze()[i] * torch.exp(MN(loc=loc.squeeze()[i], scale_tril=cov[:, i, :, :].squeeze()).log_prob(X[:, t, tt, :]))
                    logprob += torch.log(current_prob)
        return logprob
        
# Pyro version    
    
class PyroRMDN(nn.Module):
    """
    Generative model for the above RMDN.
    The class is used for training, sampling, and validation.
    """
    
    def __init__(self, input_dim, hidden_dim, output_dim, K, grid=48):
        super(PyroRMDN, self).__init__()
        # Initialise recurrent mixture density network
        self.RMDN = RMDN(input_dim, hidden_dim, output_dim, K)
        self.K = K
        self.output_dim = output_dim
        self.grid = grid        
        
    def model(self, X=None, U=None, mask=None, hidden=None, batch_size=1, validation=False):
        # Number of sequences
        N = len(U)
        # Number of time steps
        T_max = U.size(1)
        # Batch size
        b = min(N, batch_size)
        # Dimension check
        assert U.shape == (N, T_max, self.grid, self.grid)       
        # Initialise sample tensor
        x_samples = torch.zeros((b, T_max, max(mask), 2))
        # Register with Pyro
        pyro.module("PyroRMDN",self)
        # Main plate
        with pyro.plate("data", N, dim=-2):
            # Iterate through each time interval
            for t in pyro.markov(range(0, T_max)):
                # Extract mixture components
                (loc, pi, cov), hidden = self.RMDN(U[:, t, :, :], hidden)
                # If model is to be evaluated
                
                with pyro.plate('density_%d'%t, size=mask[t], dim=-1):
                    # Draw which component
                    assignment = pyro.sample('assignment_%d'%t, dist.Categorical(pi.view(-1, self.K)))
                    # Extract corresponding mean and covariance
                    _loc = loc[:, assignment, :].view(-1, 2)
                    _cov = cov[:, assignment, :, :].view(-1, 2, 2)
                    # Draw sample as multivariate normal
                    if X is None:
                        x_samples[:, t, :mask[t]] = pyro.sample('x_%d'%t, MN(loc=_loc, scale_tril=_cov),obs=None)
                    else:
                        x_samples[:, t, :mask[t]] = pyro.sample('x_%d'%t, MN(loc=_loc, scale_tril=_cov),obs=X[:, t, :mask[t], :])

        return x_samples
    
    def guide(self, X=None, U=None, mask=None, hidden=None):
        pass
    
    def get_loglikelihood(self, U, X, mask):
        return self.RMDN.get_loglikelihood(U, X, mask)


In [39]:
dataset  = RFNDataset(X_train_tensor, U_train_tensor)
dataloader= DataLoader(dataset, batch_size=1, shuffle=True, num_workers=0)

In [40]:
pyronet = PyroRMDN(input_dim=48*48, hidden_dim=128, output_dim=2, K=10)
optimizer = pyro.optim.Adam({"lr": 0.001})
svi = SVI(pyronet.model, pyronet.guide, optimizer, TraceEnum_ELBO(num_particles=1, max_plate_nesting=1))

In [41]:
pyro.clear_param_store()
num_epochs = 200
train_losses = []
val_losses = []
train_ll = []
validation_ll =  []
for i in range(num_epochs):
    hidden = pyronet.RMDN.init_hidden()
    for X_i, U_i in dataloader:
        # Set model to training
        pyronet.train()
        # Take step and update parameters
        loss = svi.step(X_i, U_i, N_t_train, hidden) /(N_t_train.sum())
        # Save current loss
        train_losses.append(loss)       
        if i%100 == 99 or i == 0:
            # Set model to evaluation
            pyronet.eval()
            # Compute and save training and validation log likelihood
            train_ll_i      = pyronet.get_loglikelihood(X=X_i, U=U_i, mask=N_t_train)
            validation_ll_i = pyronet.get_loglikelihood(X=X_val_tensor, U=U_val_tensor, mask=N_t_valid)
            train_ll.append(train_ll_i)
            validation_ll.append(validation_ll_i)
            
            print(f"Epoch: {i+1}, \tLoss: {loss:.3f}, \tTraining LL: {train_ll_i.numpy()[0]},\tValidation LL: {validation_ll_i.numpy()[0]}")
        

Epoch: 1, 	Loss: 5.102, 	Training LL: -100504.203125,	Validation LL: -52669.046875
Epoch: 100, 	Loss: 4.749, 	Training LL: -90215.3046875,	Validation LL: -47382.47265625
Epoch: 200, 	Loss: 4.745, 	Training LL: -90140.2421875,	Validation LL: -47381.4765625


In [18]:
pyronet.get_loglikelihood(U=U_test_tensor, X=X_test_tensor, mask=N_t_test)

tensor([-52036.6367])

In [ ]:
#model = torch.load('model')

### Baseline model

In [3]:
# Create input for mixture model
X_train_full = np.zeros((np.sum(N_t_train), 2))
prev_count = 0
for i, count in enumerate(N_t_train):
    X_train_full[prev_count:(prev_count+count), :] = X_train_tensor[:, i, :count, :].numpy()[0]
    prev_count+=count
    
# Create validation set
X_val_full = np.zeros((np.sum(N_t_valid), 2))
prev_count = 0
for i, count in enumerate(N_t_valid):
    X_val_full[prev_count:(prev_count+count), :] = X_val_tensor[:, i, :count, :].numpy()[0]
    prev_count+=count

# Create test set
X_test_full = np.zeros((np.sum(N_t_test), 2))
prev_count = 0
for i, count in enumerate(N_t_test):
    X_test_full[prev_count:(prev_count+count), :] = X_test_tensor[:, i, :count, :].numpy()[0]
    prev_count+=count

In [4]:
# Train GMM model 
from sklearn.mixture import GaussianMixture

# Initialise and fit model
baseline = GaussianMixture(n_components=10).fit(X_train_full)

# Compute loglikelihood
baseline_ll_train = baseline.score(X_train_full) * np.sum(N_t_train)
baseline_ll_validation = baseline.score(X_val_full) * np.sum(N_t_valid)
baseline_ll_test = baseline.score(X_test_full) * np.sum(N_t_test)
print(f'Baseline training LL:\t{baseline_ll_train:.4f}')
print(f'Baseline validation LL:\t{baseline_ll_validation:.4f}')
print(f'Baseline testing LL:\t{baseline_ll_test:.4f}')

Baseline training LL:	-70574.1042
Baseline validation LL:	-36432.0634
Baseline testing LL:	-36508.0325
